#### Festlegung für die Darstellung von interaktiven Graphiken innerhalb des Notebooks

In [ ]:
%pylab inline

#### Import der notwendigen Pakete und Module

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.constants as const
import scipy
from scipy.io import wavfile
from pylab import *
from scipy.fftpack import fft, ifft
from IPython.display import Audio

import functions.dsvorg as do
import functions.data_resample as resample
import functions.bsfilter as bsfilter
import functions.classes as dc
import functions.tproc_new as tp

[Table of Contents](table_of_contents.ipynb)

In [ ]:
import functions.PrettyTable as pt

##### Protokoll
## Signale, akustische Ereignisse

<table align="left" width="400">
    <colgroup>
       <col span="1" style="width: 50%;">
       <col span="1" style="width: 50%;">
    </colgroup>
  <tr>
    <th>Name</th>
    <th>Vorname</th> 
  </tr>
  <tr>
    <td> Flach</td>
    <td> Gudrun</td>
  </tr>
      <tr>
    <td> Mustermann</td>
    <td> Max</td>
  </tr>

</table>


#### Hinweis:
In dem Verzeichnis `sound` sind die für das Praktikum erforderlichen Signale enthalten. Alle im Rahmen der Experimente erstellten Signale werden unter den angegebenen Namen ebenfalls in diesem Verzeichnis abgespeichert.

In [ ]:
info_data = []
head = ['Dateiname', 'Abtastrate', 'Datenformat', 'Anz. ATW', 'Maximum', 'Minimum']

### Signalinformationen
Für die korrekte Verarbeitung digitaler Signale sind eine Reihe von Informationen erforderlich, die in dem Dateiheader gehalten werden.  
### <font color="#008000">Aufgabe 1</font>
Untersuchen Sie die aufgeführten Signale und berechnen Sie die Signaldauer. 

#### <font color="#FF0000">Eingabezelle 1</font>

In [ ]:
#file = 'gaense'
#file = 'glocken'
file = 'mann'
#file = 'rauschen'
#file = 'traktor_fahrt'
#file = 'traktor_stand'
#file = 'wasser'

In [ ]:
info_data.append(do.get_info(file))
pt.PrettyTable(info_data, head)

##### Protokoll
### Signaldauer

<table align="left" width="400">
    <colgroup>
       <col span="1" style="width: 50%;">
       <col span="1" style="width: 50%;">
    </colgroup>
  <tr>
    <th>Datei</th>
    <th>Signaldauer in ms</th> 
  </tr>
  <tr>
    <td> gaense</td>
    <td> </td>
  </tr>
  <tr>
    <td> glocken</td>
    <td> </td>
  </tr>
  <tr>
    <td> mann</td>
    <td> </td>
  </tr>
  <tr>
    <td> rauschen</td>
    <td> </td>
  </tr>
  <tr>
    <td> traktor_fahrt</td>
    <td> </td>
  </tr>
  <tr>
    <td> traktor_stand</td>
    <td> </td>
  </tr>
  <tr>
    <td> wasser</td>
    <td> </td>
  </tr>

</table>


### Normierung
Die vorliegenden Daten unterscheiden sich hinsichtlich einiger Kriterien. Um Daten miteinander vergleichen zu können, sollten sie normiert werden. Zunächst soll eine Normierung auf den maximalen Betragswert des Signals vorgenommen werden.

### <font color="#008000">Aufgabe 2</font>
Sie sehen und hören ein nichtnormiertes und ein nichtnormiertes Signal. Vergleichen Sie die Darstellungen und die 3 Audioausgaben. Beschreiben und erklären Sie den beobachteten Effekt in der **Tabelle Normierung**!

In [ ]:
fs, data = do.load_data_o('craco_o')
fs, data_norm = do.load_data('craco_o')
n = np.linspace(0, len(data)-1, len(data))
plt.figure(figsize=(15,5))
subplot(221)
plt.plot(n, data)
plt.title('Signal original')
subplot(222)
plt.plot(n, data_norm)
plt.title('Signal normiert')
plt.savefig('a2-2.jpg')
plt.show()

#### Audioausgabe 1
Sie hören zunächst das originale und unmittelbar anschließend das normierte Signal.

In [ ]:
data_out = np.append(data, data_norm)
Audio(data = data_out, rate = fs)

#### Audioausgabe 2
Sie hören das nicht normierte Signal. 

In [ ]:
Audio(data = data, rate = fs)

#### Audioausgabe 3
Sie hören das normierte Signal.  

In [ ]:
Audio(data = data_norm, rate = fs)

##### Protokoll
#### Aufgabe 2 - Tabelle Normierung
<table>
    <colgroup>
       <col span="1" style="width: 12%;">
       <col span="1" style="width: 40%;">
       <col span="1" style="width: 40%;">
    </colgroup>
  <tr>
    <th>Audioausgabe</th>
    <th>Beobachtung</th> 
    <th>Erklärung</th>
  </tr>
  <tr>
    <td>nicht normiert + normiert</td>
    <td> </td>
    <td> </td>
  </tr>
  <tr>
    <td>nicht normiert</td>
    <td> </td>
    <td> </td>
  </tr>
  <tr>
    <td>normiert</td>
    <td> </td>
    <td> </td>
  </tr>
</table>
<img src = 'a2-2.jpg'>

### Veränderung der Dynamik
Zur Veränderung der Dynamik stehen 4 Verfahren zur Verfügung (`komp`, `limit`, `expand` und `gate`, - Eingabezelle 3, Variable `typ` ), die in der Versuchsanleitung erläutert sind.

### <font color="#008000">Aufgabe 3</font>
Untersuchen Sie den Einfluss dieser Verfahren zunächst anhand eines Sinustones (Dauer 1s, Frequenz 800 Hz, Abtastfrequenz 16 kHz). 

- Testen Sie für jedes Verfahren 2 unterschiedliche Parameterkombinationen (`threshold`, `ratio`).
- Tragen Sie die wahrgenommenen Veränderungen in der graphischen und akustischen Wahrnehmung in der **Tabelle Dynamik** ein.

#### <font color="#FF0000">Eingabezelle 3</font>

In [ ]:
threshold=0.2
ratio=0.3
typ='komp'

In [ ]:
sig = dc.sound.Sound()
testsig = sig.generate_tone(800)
weight = dc.dynamic.Dynamic(threshold=threshold, ratio=ratio, typ=typ)
dynsig = do.demo_komp(testsig, weight)

Bei der Audioausgabe hören Sie zunächst das Originalsignal (blau) und im Anschluss das mit dem Kompressionsverfahren behandelte Signal (grün). 

In [ ]:
data = np.append(testsig, dynsig)
Audio(data = data, rate = 16000)

##### Protokoll
#### Aufgabe 3 - Tabelle Dynamik
<table>
    <colgroup>
       <col span="1" style="width: 10%;">
       <col span="1" style="width: 10%;">
       <col span="1" style="width: 10%;">
       <col span="1" style="width: 35%;">
       <col span="1" style="width: 35%;">
    </colgroup>
  <tr>
    <th>Verfahren</th>
    <th>`threshold`</th> 
    <th>`ratio`</th>
    <th>Veränderungen in der Signalform</th>
    <th>Veränderung im Klang</th>
  </tr>
  <tr>
    <th>Expander</th>
    <th> </th> 
    <th> </th>
    <th> </th>
    <th> </th>
  </tr>
  <tr>
    <th>Expander</th>
    <th> </th> 
    <th> </th>
    <th> </th>
    <th> </th>
  </tr>
  <tr>
    <th>Kompressor</th>
    <th> </th> 
    <th> </th>
    <th> </th>
    <th> </th>
  </tr>
  <tr>
    <th>Kompressor</th>
    <th> </th> 
    <th> </th>
    <th> </th>
    <th> </th>
  </tr>
  <tr>
    <th>Limiter</th>
    <th> </th> 
    <th> </th>
    <th> </th>
    <th> </th>
  </tr>
  <tr>
    <th>Limiter</th>
    <th> </th> 
    <th> </th>
    <th> </th>
    <th> </th>
  </tr>
  <tr>
    <th>Gate</th>
    <th> </th> 
    <th> </th>
    <th> </th>
    <th> </th>
  </tr>
  <tr>
    <th>Gate</th>
    <th> </th> 
    <th> </th>
    <th> </th>
    <th> </th>
  </tr>
</table>

#### <font color="#008000">Aufgabe 4</font>
Wenden Sie die Kompressionsverfahren mit den in der **Tabelle Anwendung Dynamikkompression** enthaltenen Parametern auf die folgenden Signale an und beschreiben Sie die Veränderung im Klang.
- `dsv1_mann.wav` (1)
- `dsv1_kind.wav` (2)

#### <font color="#FF0000">Eingabezelle 4</font>

In [ ]:
file = 'dsv1_mann'
threshold=0.2
ratio=2
typ='expand'

In [ ]:
weight = dc.dynamic.Dynamic(threshold=threshold, ratio=ratio, typ=typ)
o_sig, fs = do.read_sig(file)
d_sig = do.sig_komp(o_sig, weight)
n = np.linspace(0, (len(o_sig)-1)/fs,len(o_sig))
plt.figure(figsize=(12,3))
subplot(121)
plot(n, o_sig)
title('Originalsignal')
subplot(122)
plot(n, d_sig)
title('Signal mit Dynamikkompression')
data = np.append(o_sig, d_sig)
Audio(data = data, rate = fs)

##### Protokoll
#### Aufgabe 4 - Tabelle Anwendung Dynamikkompression
<table>
    <colgroup>
       <col span="1" style="width: 10%;">
       <col span="1" style="width: 10%;">
       <col span="1" style="width: 10%;">
       <col span="1" style="width: 35%;">
       <col span="1" style="width: 35%;">
    </colgroup>
  <tr>
    <th>Verfahren</th>
    <th>`threshold`</th> 
    <th>`ratio`</th>
    <th>Veränderungen in der Signalform</th>
    <th>Veränderung im Klang</th>
  </tr>
  <tr>
    <th>Kompressor</th>
    <th>0.3 </th> 
    <th>0.2 </th>
    <th> </th>
    <th> </th>
  </tr>
  <tr>
    <th>Limiter</th>
    <th>0.4 </th> 
    <th>- </th>
    <th> </th>
    <th> </th>
  </tr>
  <tr>
    <th>Expander</th>
    <th>0.2 </th> 
    <th> 2</th>
    <th> </th>
    <th> </th>
  </tr>
  <tr>
    <th>Gate</th>
    <th> 0.2</th> 
    <th> -</th>
    <th> </th>
    <th> </th>
  </tr>
</table>

In [ ]:
nb_in = 'D:\\Dokumente\\Lehre\\module\\DSV-MI\\notebooks\\DSM4\\DSVMI_part2.ipynb'
nb_out = 'D:\\Dokumente\\Lehre\\module\\DSV-MI\\notebooks\\DSM4\\Protokoll_part2.ipynb'
tp.protokoll(nb_in,nb_out)